In [ ]:
import os
os._exit(00)

In [ ]:
pip install PyPDF2


In [ ]:
pip install transformers

In [ ]:
pip install numpy

In [ ]:
pip install rouge

In [ ]:
input_folder = "/content/drive/MyDrive/Headnote Judgment"
output_folder_text = "/content/drive/MyDrive/Headnote Judgment/Judgment"
output_folder_headnotes = "/content/drive/MyDrive/Headnote Judgment/Summary"

## **Run to extract the file and its headnote and keeping it to different folders**

In [ ]:
import os
from PyPDF2 import PdfReader
import shutil
import re

# Define input and output folders
input_folder = "/content/drive/MyDrive/Headnote Judgment"
output_folder_text = "/content/drive/MyDrive/Headnote Judgment/Judgment"
output_folder_headnotes = "/content/drive/MyDrive/Headnote Judgment/Summary"

# Create output folders if they don't exist
os.makedirs(output_folder_text, exist_ok=True)
os.makedirs(output_folder_headnotes, exist_ok=True)

# Function to extract text and headnote from PDF file
def extract_text_and_headnote_from_pdf(pdf_path):
    pdf_reader = PdfReader(pdf_path)
    text = ""
    for page_num in range(len(pdf_reader.pages)):
        text += pdf_reader.pages[page_num].extract_text()

    text=text.lower()

    # Extract headnote using regular expression pattern
    headnote_search = re.search(r"headnote:(.*?)judgment:", text, re.DOTALL | re.IGNORECASE)
    if headnote_search:
        # Extract the headnote and remove leading/trailing spaces
        headnote = headnote_search.group(1).strip()
    else:
        headnote = "Headnote not found."

    return text, headnote

# Iterate through PDF files in the input folder
for filename in os.listdir(input_folder):
    if filename.lower().endswith(".pdf"):
        pdf_path = os.path.join(input_folder, filename)
        # Extract text and headnote from the PDF
        extracted_text, headnote = extract_text_and_headnote_from_pdf(pdf_path)

        # Write extracted text to the output text folder
        output_text_path = os.path.join(output_folder_text, f"{len(os.listdir(output_folder_text)) + 1}.txt")
        with open(output_text_path, "w", encoding="utf-8") as output_file:
            output_file.write(extracted_text)

        # Write headnote to the output headnotes folder
        output_headnote_path = os.path.join(output_folder_headnotes, f"{len(os.listdir(output_folder_headnotes)) + 1}.txt")
        with open(output_headnote_path, "w", encoding="utf-8") as headnote_output_file:
            headnote_output_file.write(headnote)

print("Extraction completed.")



Extraction completed.


## **InLegalbert**

In [ ]:
import os
import pandas as pd
from transformers import AutoTokenizer, AutoModel
from rouge import Rouge
import numpy as np

# Load the InLegalBERT model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("law-ai/InLegalBERT")
model = AutoModel.from_pretrained("law-ai/InLegalBERT")

# Function to read text from a file
def read_text_from_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.read()

# Function to generate summary using InLegalBERT
def generate_summary(text):
  tokenizer.max_len = model.config.max_position_embeddings

  # Encode the input text
  encoded_input = tokenizer(text, return_tensors="pt", truncation=True)
  output = model(**encoded_input)
  embeddings = output.pooler_output

# Making a summary using these embeddings
# Step 1: Compute sentence embeddings
  sentences = text.split('. ')  # Split text into sentences

# Calculate the desired number of sentences for the summary based on the percentage
  percentage = 0.5  # Adjust the desired percentage as needed
  num_sentences = int(len(sentences) * percentage)

  if len(sentences) < num_sentences:
      num_sentences = len(sentences)

  sentence_embeddings = []
  for sentence in sentences:
      encoded_input = tokenizer(sentence, return_tensors="pt", truncation=True)
      output = model(**encoded_input)
      sentence_embedding = output.pooler_output.squeeze(dim=0)
      sentence_embeddings.append(sentence_embedding.detach().numpy())

# Step 2: Compute similarity scores
  similarity_scores = []
  for i, embedding_i in enumerate(sentence_embeddings):
      similarity_scores_i = []
      for j, embedding_j in enumerate(sentence_embeddings):
          similarity = np.dot(embedding_i, embedding_j) / (
              np.linalg.norm(embedding_i) * np.linalg.norm(embedding_j)
          )
          similarity_scores_i.append(similarity)
      similarity_scores.append(similarity_scores_i)

  similarity_scores = np.array(similarity_scores)

# Step 3: Select top sentences
  summary_indices = np.argsort(-similarity_scores.sum(axis=1))[:num_sentences]
  summary_sentences = [sentences[i] for i in summary_indices]
  InLegal_summary = '. '.join(summary_sentences)
  return InLegal_summary

# Path to the judgment and summary folders
judgment_folder = '/content/drive/MyDrive/Headnote Judgment/Judgment'
summary_folder = '/content/drive/MyDrive/Headnote Judgment/Summary'

# Initialize lists to store evaluation metrics
file_names = []
rouge1_scores = []
rouge2_scores = []
rougel_scores = []
precision_scores = []
recall_scores = []
f1_scores = []

# List all files in the judgment folder
judgment_files = os.listdir(judgment_folder)

# Initialize Rouge object for evaluation
rouge = Rouge()

# Iterate through judgment files and generate summaries
for judgment_file in judgment_files:
    # Extract text from the judgment file
    judgment_file_path = os.path.join(judgment_folder, judgment_file)
    judgment_text = read_text_from_file(judgment_file_path)

    # Find corresponding summary file in the summary folder
    summary_file_path = os.path.join(summary_folder, judgment_file)

    # Read the reference summary from the summary file
    reference_summary = read_text_from_file(summary_file_path)

    # Generate summary using InLegalBERT
    generated_summary = generate_summary(judgment_text)

    # Calculate ROUGE scores
    rouge_scores = rouge.get_scores(generated_summary, reference_summary)[0]

    # Store the metrics in lists
    file_names.append(judgment_file)
    rouge1_scores.append(rouge_scores['rouge-1']['f'])
    rouge2_scores.append(rouge_scores['rouge-2']['f'])
    rougel_scores.append(rouge_scores['rouge-l']['f'])
    precision_scores.append(rouge_scores['rouge-l']['p'])
    recall_scores.append(rouge_scores['rouge-l']['r'])
    f1_scores.append(rouge_scores['rouge-l']['f'])

# Create a DataFrame from the lists
data = {
    'File Name': file_names,
    'InLegal ROUGE-1': rouge1_scores,
    'InLegal ROUGE-2': rouge2_scores,
    'InLegal ROUGE-L': rougel_scores,
    'InLegal Precision': precision_scores,
    'InLegal Recall': recall_scores,
    'InLegal F1 Score': f1_scores
}

df = pd.DataFrame(data)

# Save the DataFrame to an Excel file
excel_file_path = '/content/drive/MyDrive/Headnote Judgment/new_evaluation_results.xlsx'
df.to_excel(excel_file_path, index=False)

print(f"Evaluation metrics saved to {excel_file_path}")


Evaluation metrics saved to /content/drive/MyDrive/Headnote Judgment/new_evaluation_results.xlsx


In [ ]:
rouge1_scores

[]

## **LegalPegasus**

In [ ]:
pip install sentencepiece

In [ ]:
import os
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from rouge import Rouge

# Load the Legal Pegasus model and tokenizer
legal_pegasus_tokenizer = AutoTokenizer.from_pretrained("nsi319/legal-pegasus")
legal_pegasus_model = AutoModelForSeq2SeqLM.from_pretrained("nsi319/legal-pegasus")

# Function to read text from a file
def read_text_from_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.read()

# Function to generate summary using Legal Pegasus
def generate_legal_pegasus_summary(text):
    input_tokenized = legal_pegasus_tokenizer.encode(text, return_tensors='pt', max_length=1024, truncation=True)
    summary_ids = legal_pegasus_model.generate(input_tokenized,
                                              num_beams=9,
                                              no_repeat_ngram_size=3,
                                              length_penalty=2.0,
                                              min_length=150,
                                              max_length=250,
                                              early_stopping=True)
    summary_legal_pegasus = [legal_pegasus_tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in summary_ids][0]
    return summary_legal_pegasus

# Path to the judgment and summary folders
judgment_folder = '/content/drive/MyDrive/Headnote Judgment/Judgment'
summary_folder = '/content/drive/MyDrive/Headnote Judgment/Summary'

# Load the existing DataFrame from the Excel file
excel_file_path = '/content/drive/MyDrive/Headnote Judgment/new_evaluation_results.xlsx'
df = pd.read_excel(excel_file_path)

# Initialize lists to store Legal Pegasus evaluation metrics
legal_pegasus_rouge1 = []
legal_pegasus_rouge2 = []
legal_pegasus_rougeL = []
legal_pegasus_precision = []
legal_pegasus_recall = []
legal_pegasus_f1_scores = []

# Initialize Rouge object for evaluation
rouge = Rouge()

# List all files in the judgment folder
judgment_files = os.listdir(judgment_folder)

# Iterate through judgment files and generate summaries using Legal Pegasus
for judgment_file in judgment_files:
    # Extract text from the judgment file
    judgment_file_path = os.path.join(judgment_folder, judgment_file)
    judgment_text = read_text_from_file(judgment_file_path)

    # Find corresponding summary file in the summary folder
    summary_file_path = os.path.join(summary_folder, judgment_file)

    # Read the reference summary from the summary file
    reference_summary = read_text_from_file(summary_file_path)

    # Generate summary using Legal Pegasus
    generated_summary_legal_pegasus = generate_legal_pegasus_summary(judgment_text)

    # Calculate ROUGE scores for Legal Pegasus
    rouge_scores_legal_pegasus = rouge.get_scores(generated_summary_legal_pegasus, reference_summary)[0]

    # Append metrics to the respective lists
    legal_pegasus_rouge1.append(rouge_scores_legal_pegasus['rouge-1']['f'])
    legal_pegasus_rouge2.append(rouge_scores_legal_pegasus['rouge-2']['f'])
    legal_pegasus_rougeL.append(rouge_scores_legal_pegasus['rouge-l']['f'])
    legal_pegasus_precision.append(rouge_scores_legal_pegasus['rouge-l']['p'])
    legal_pegasus_recall.append(rouge_scores_legal_pegasus['rouge-l']['r'])
    legal_pegasus_f1_scores.append(rouge_scores_legal_pegasus['rouge-l']['f'])

# Add Legal Pegasus evaluation metrics to the DataFrame
df['LegalPegasus_ROUGE1'] = legal_pegasus_rouge1
df['LegalPegasus_ROUGE2'] = legal_pegasus_rouge2
df['LegalPegasus_ROUGEL'] = legal_pegasus_rougeL
df['LegalPegasus_Precision'] = legal_pegasus_precision
df['LegalPegasus_Recall'] = legal_pegasus_recall
df['LegalPegasus'] = legal_pegasus_f1_scores


# Save the updated DataFrame to the same Excel file
df.to_excel(excel_file_path, index=False)

print(f"Legal Pegasus evaluation metrics added to {excel_file_path}")


Legal Pegasus evaluation metrics added to /content/drive/MyDrive/Headnote Judgment/new_evaluation_results.xlsx


## **Lex,Luhn,Text**

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
pip install sumy

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
import os
import pandas as pd
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lex_rank import LexRankSummarizer
from sumy.summarizers.luhn import LuhnSummarizer
from sumy.summarizers.text_rank import TextRankSummarizer
from sumy.utils import get_stop_words
from rouge import Rouge

# Function to read text from a file
def read_text_from_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.read()

# Function to perform summarization using LexRank
def lex_rank_summarize(text, num_sentences):
    parser = PlaintextParser.from_string(text, Tokenizer("english"))
    summarizer = LexRankSummarizer()
    summarizer.stop_words = get_stop_words("english")
    summary = summarizer(parser.document, sentences_count=num_sentences)
    return " ".join([str(sentence) for sentence in summary])

# Function to perform summarization using Luhn
def luhn_summarize(text, num_sentences):
    parser = PlaintextParser.from_string(text, Tokenizer("english"))
    summarizer = LuhnSummarizer()
    summarizer.stop_words = get_stop_words("english")
    summary = summarizer(parser.document, sentences_count=num_sentences)
    return " ".join([str(sentence) for sentence in summary])

# Function to perform summarization using TextRank
def text_rank_summarize(text, num_sentences):
    parser = PlaintextParser.from_string(text, Tokenizer("english"))
    summarizer = TextRankSummarizer()
    summarizer.stop_words = get_stop_words("english")
    summary = summarizer(parser.document, sentences_count=num_sentences)
    return " ".join([str(sentence) for sentence in summary])

# Path to the judgment and summary folders
judgment_folder = '/content/drive/MyDrive/Headnote Judgment/Judgment'
summary_folder = '/content/drive/MyDrive/Headnote Judgment/Summary'

# Load the existing DataFrame from the Excel file
excel_file_path = '/content/drive/MyDrive/Headnote Judgment/new_evaluation_results.xlsx'
df = pd.read_excel(excel_file_path)

# Initialize lists to store LexRank, Luhn, and TextRank evaluation metrics
lex_rank_rouge1 = []
lex_rank_rouge2 = []
lex_rank_rougeL = []
lex_rank_precision = []
lex_rank_recall = []
lex_rank_f1_scores = []
luhn_rouge1 = []
luhn_rouge2 = []
luhn_rougeL = []
luhn_precision = []
luhn_recall = []
luhn_f1_scores = []
text_rank_rouge1 = []
text_rank_rouge2 = []
text_rank_rougeL = []
text_rank_precision = []
text_rank_recall = []
text_rank_f1_scores = []

# Initialize Rouge object for evaluation
rouge = Rouge()

# List all files in the judgment folder
judgment_files = os.listdir(judgment_folder)

# Iterate through judgment files and generate summaries using LexRank, Luhn, and TextRank
for judgment_file in judgment_files:
    # Extract text from the judgment file
    judgment_file_path = os.path.join(judgment_folder, judgment_file)
    judgment_text = read_text_from_file(judgment_file_path)

    num_sentences = int(0.5 * len(judgment_text.split('. ')))

    # Find corresponding summary file in the summary folder
    summary_file_path = os.path.join(summary_folder, judgment_file)
    # Read the reference summary from the summary file
    reference_summary = read_text_from_file(summary_file_path)

    # LexRank Summarization
    lex_summary = lex_rank_summarize(judgment_text, num_sentences)
    lex_summary= lex_summary.replace(u'\xa0',' ')
    lex_summary= lex_summary.replace(u'\xad',' ')
    lex_summary= lex_summary.replace(u'\n',' ')
    lex_summary= lex_summary.replace(u'...',' ')
    lex_scores = rouge.get_scores(lex_summary, reference_summary)[0]
    lex_rank_rouge1.append(lex_scores['rouge-1']['f'])
    lex_rank_rouge2.append(lex_scores['rouge-2']['f'])
    lex_rank_rougeL.append(lex_scores['rouge-l']['f'])
    lex_rank_precision.append(lex_scores['rouge-l']['p'])
    lex_rank_recall.append(lex_scores['rouge-l']['r'])
    lex_rank_f1_scores.append((lex_scores['rouge-l']['f']))

    # Luhn Summarization
    luhn_summary = luhn_summarize(judgment_text, num_sentences)
    luhn_summary = luhn_summary.replace(u'\xa0',' ')
    luhn_summary = luhn_summary.replace(u'\xad',' ')
    luhn_summary = luhn_summary.replace(u'\n',' ')
    luhn_summary = luhn_summary.replace(u'...',' ')
    luhn_scores = rouge.get_scores(luhn_summary, reference_summary)[0]
    luhn_rouge1.append(luhn_scores['rouge-1']['f'])
    luhn_rouge2.append(luhn_scores['rouge-2']['f'])
    luhn_rougeL.append(luhn_scores['rouge-l']['f'])
    luhn_precision.append(luhn_scores['rouge-l']['p'])
    luhn_recall.append(luhn_scores['rouge-l']['r'])
    luhn_f1_scores.append((luhn_scores['rouge-l']['f']))

    # TextRank Summarization
    text_rank_summary = text_rank_summarize(judgment_text, num_sentences)
    text_rank_summary = text_rank_summary.replace(u'\xa0',' ')
    text_rank_summary = text_rank_summary.replace(u'\xad',' ')
    text_rank_summary = text_rank_summary.replace(u'\n',' ')
    text_rank_summary = text_rank_summary.replace(u'...',' ')
    text_rank_scores = rouge.get_scores(text_rank_summary, reference_summary)[0]
    text_rank_rouge1.append(text_rank_scores['rouge-1']['f'])
    text_rank_rouge2.append(text_rank_scores['rouge-2']['f'])
    text_rank_rougeL.append(text_rank_scores['rouge-l']['f'])
    text_rank_precision.append(text_rank_scores['rouge-l']['p'])
    text_rank_recall.append(text_rank_scores['rouge-l']['r'])
    text_rank_f1_scores.append((text_rank_scores['rouge-l']['f']))

# Add LexRank, Luhn, and TextRank evaluation metrics to the DataFrame
df['LexRank_ROUGE1'] = lex_rank_rouge1
df['LexRank_ROUGE2'] = lex_rank_rouge2
df['LexRank_ROUGEL'] = lex_rank_rougeL
df['LexRank_Precision'] = lex_rank_precision
df['LexRank_Recall'] = lex_rank_recall
df['LexRank_F1'] = lex_rank_f1_scores
df['Luhn_ROUGE1'] = luhn_rouge1
df['Luhn_ROUGE2'] = luhn_rouge2
df['Luhn_ROUGEL'] = luhn_rougeL
df['Luhn_Precision'] = luhn_precision
df['Luhn_Recall'] = luhn_recall
df['Luhn_F1'] = luhn_f1_scores
df['TextRank_ROUGE1'] = text_rank_rouge1
df['TextRank_ROUGE2'] = text_rank_rouge2
df['TextRank_ROUGEL'] = text_rank_rougeL
df['TextRank_Precision'] = text_rank_precision
df['TextRank_Recall'] = text_rank_recall
df['TextRank_F1'] = text_rank_f1_scores

# Save the updated DataFrame to the same Excel file
df.to_excel(excel_file_path, index=False)

print(f"LexRank, Luhn, and TextRank evaluation metrics added to {excel_file_path}")


LexRank, Luhn, and TextRank evaluation metrics added to /content/drive/MyDrive/Headnote Judgment/new_evaluation_results.xlsx


In [ ]:
text_rank_summary

'abdulla kunhi and b.l. abdul   vs union of india and ors., state of   on 23 january, 1991 equivalent citations: 1991 air 574, 1991 scr (1) 102 author: k shetty bench: ray, b.c. (j), sharma, l.m. abdulla kunhi and b.l. sharma, l.m. (j) verma, jagdish saran (j) citation: 1991 air  574            1991 scr  (1) 102 1991 scc  (1) 476        jt 1991 (1)   216 1991 scale  (1)58 citator info : rf&e       1992 sc2161  (9) act: conservation  of  foreign exchange  and  prevention  of smuggling  activities act, 1974--sections 3,8, 10  and  11-- detention   order   confirmed   before   consideration    of representation   of  detenu--whether   valid--representation received by considered by government after receiving  report of advisory board--whether valid. constitution  of india 1950: articles 22(4)  and  (5)-- preventive detention--rights of detenu--what are. abdulla kunhi and b.l. mahajan, b. parthasarthy. p. parmeswaran and m. veerappa for the respondents. the detenu made representation to th

## T5 **Model**

In [ ]:
pip install transformers

In [ ]:
pip install sentencepiece

In [ ]:
import os
import pandas as pd
from transformers import T5Tokenizer, T5ForConditionalGeneration
from rouge import Rouge

# Load the model and tokenizer
tokenizer = T5Tokenizer.from_pretrained('t5-base')
model = T5ForConditionalGeneration.from_pretrained('t5-base')

rouge = Rouge()

def read_text_from_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.read()

def generate_summary_T5(input_text):
    input_ids = tokenizer.encode(input_text, return_tensors='pt', max_length=512, truncation=True)
    #total_tokens = len(input_ids[0])
    #target_length = total_tokens // 2
    summary_ids = model.generate(input_ids, max_length=128, min_length=32, length_penalty=2.0, num_beams=4, early_stopping=True)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

# Path to the judgment and summary folders
judgment_folder = '/content/drive/MyDrive/Headnote Judgment/Judgment'
summary_folder = '/content/drive/MyDrive/Headnote Judgment/Summary'
excel_file_path = '/content/drive/MyDrive/Headnote Judgment/new_evaluation_results.xlsx'

# Load the existing DataFrame from the Excel file
df = pd.read_excel(excel_file_path)

# Initialize lists to store evaluation metrics
rouge1_scores = []
rouge2_scores = []
rougeL_scores = []
precision_scores = []
recall_scores = []
f1_scores = []

# List all files in the judgment folder
judgment_files = os.listdir(judgment_folder)

# Iterate through judgment files and generate summaries, calculate ROUGE scores
for judgment_file in judgment_files:
    # Extract text from the judgment file
    judgment_file_path = os.path.join(judgment_folder, judgment_file)
    judgment_text = read_text_from_file(judgment_file_path)

    # Find corresponding summary file in the summary folder
    summary_file_path = os.path.join(summary_folder, judgment_file)

    # Read the reference summary from the summary file
    reference_summary = read_text_from_file(summary_file_path)

    # Generate summary
    generated_summary = generate_summary_T5(judgment_text)

    # Calculate ROUGE scores
    rouge_scores = rouge.get_scores(generated_summary, reference_summary)[0]

    # Append metrics to the respective lists
    rouge1_scores.append(rouge_scores['rouge-1']['f'])
    rouge2_scores.append(rouge_scores['rouge-2']['f'])
    rougeL_scores.append(rouge_scores['rouge-l']['f'])
    precision_scores.append(rouge_scores['rouge-l']['p'])
    recall_scores.append(rouge_scores['rouge-l']['r'])
    f1_scores.append(rouge_scores['rouge-l']['f'])

# Add evaluation metrics to the DataFrame
df['T5 ROUGE-1'] = rouge1_scores
df['T5 ROUGE-2'] = rouge2_scores
df['T5 ROUGE-L'] = rougeL_scores
df['T5 Precision'] = precision_scores
df['T5 Recall'] = recall_scores
df['T5 F1 Score'] = f1_scores

# Save the updated DataFrame to the same Excel file
df.to_excel(excel_file_path, index=False)

print(f"Evaluation metrics added to {excel_file_path}")



/usr/local/lib/python3.10/dist-packages/transformers/models/t5/tokenization_t5.py:238: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


Evaluation metrics added to /content/drive/MyDrive/Headnote Judgment/new_evaluation_results.xlsx


In [ ]:
generated_summary

'k shetty bench: k.m. abdulla kunhi and b.l. abdul khader vs. union of india and ors., state of karnataka on 23 january, 1991. ray, b.c. (j) kania, m.h. sharma, l.m. (j) verma, jagdish saran (j)'

## **BART**

In [ ]:
import os
import pandas as pd
from transformers import BartTokenizer, BartForConditionalGeneration
from rouge import Rouge

# Initialize BART model and tokenizer
model = BartForConditionalGeneration.from_pretrained('sshleifer/distilbart-cnn-12-6')
tokenizer = BartTokenizer.from_pretrained('sshleifer/distilbart-cnn-12-6')

# Path to the judgment and summary folders
judgment_folder = '/content/drive/MyDrive/Headnote Judgment/Judgment'
summary_folder = '/content/drive/MyDrive/Headnote Judgment/Summary'
excel_file_path = '/content/drive/MyDrive/Headnote Judgment/new_evaluation_results.xlsx'
# Load the existing DataFrame from the Excel file
df = pd.read_excel(excel_file_path)

# Initialize lists to store BART evaluation metrics
bart_rouge1 = []
bart_rouge2 = []
bart_rougel = []
bart_precision = []
bart_recall = []

# Initialize Rouge object for evaluation
rouge = Rouge()

# Iterate through judgment files and generate BART summaries
for judgment_file in os.listdir(judgment_folder):
    # Read judgment text from file
    with open(os.path.join(judgment_folder, judgment_file), 'r', encoding='utf-8') as file:
        judgment_text = file.read()

    # Generate BART summary
    inputs = tokenizer([judgment_text], truncation=True, return_tensors='pt')
    summary_ids = model.generate(inputs['input_ids'], num_beams=4, early_stopping=False, min_length=0, max_length=1024)
    bart_summary = ([tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True) for g in summary_ids])

    # Read corresponding reference summary from file
    with open(os.path.join(summary_folder, judgment_file), 'r', encoding='utf-8') as file:
        reference_summary = file.read()

    # Calculate ROUGE scores for BART summary
    rouge_scores_bart = rouge.get_scores(bart_summary[0], reference_summary)[0]

    # Append metrics to the respective lists
    bart_rouge1.append(rouge_scores_bart['rouge-1']['f'])
    bart_rouge2.append(rouge_scores_bart['rouge-2']['f'])
    bart_rougel.append(rouge_scores_bart['rouge-l']['f'])
    bart_precision.append(rouge_scores_bart['rouge-l']['p'])
    bart_recall.append(rouge_scores_bart['rouge-l']['r'])

# Add evaluation metrics to the DataFrame
df['BART_ROUGE1'] = rouge1_scores
df['BART_ROUGE2'] = rouge2_scores
df['BART_ROUGEL'] = rougeL_scores
df['BART_Precision'] = precision_scores
df['BART_Recall'] = recall_scores

# Save the BART evaluation metrics to the Excel file
df.to_excel(excel_file_path, index=False)

print(f"BART evaluation metrics added to {excel_file_path}")


BART evaluation metrics added to /content/drive/MyDrive/Headnote Judgment/new_evaluation_results.xlsx


In [ ]:
bart_summary

['supreme court of india: "Every day\'s delay must be explained" "Every hour\'s delay, every second\'s delay should not be made," says judge. State seeking condonation of delay is given the power to condone delay by enacting section 51 of the indian limitation act of 1963. State sought to be treated with\'sufficient cause\' of delay. Appeal preferred by state of jammu & kashmir arising out of a decision enhancing. compensation in respect of acquisition of. lands for a public purpose to the extent of nearly 14 lakhsrupees by. making an upward revision of the order of 800% (from. 1000 per. 1000 per kanal to rs.8000 per.']

## **LetSum not working** **bold text**

In [ ]:
import os
import pandas as pd
import nltk
import numpy as np
import re
from scipy.sparse.linalg import svds
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from rouge import Rouge

# Load NLTK stopwords
nltk.download('punkt')
nltk.download('stopwords')
stop_words = set(nltk.corpus.stopwords.words('english'))

# Function to normalize documents
def normalize_document(doc):
    doc = re.sub(r'[^a-zA-Z\s]', '', doc, re.I|re.A)
    doc = doc.lower()
    doc = doc.strip()
    tokens = nltk.word_tokenize(doc)
    filtered_tokens = [token for token in tokens if token not in stop_words]
    doc = ' '.join(filtered_tokens)
    return doc

# Function to perform LSA
def perform_lsa(sentences, num_topics=3, sv_threshold=0.5, percentage=0.5):
    norm_sentences = [normalize_document(sentence) for sentence in sentences]
    tv = TfidfVectorizer(min_df=0., max_df=1., use_idf=True)
    dt_matrix = tv.fit_transform(norm_sentences)
    dt_matrix = dt_matrix.toarray()
    vocab = tv.get_feature_names_out()
    td_matrix = dt_matrix.T
    print(td_matrix.shape)
    pd.DataFrame(np.round(td_matrix, 2), index=vocab).head(10)

    u, s, vt = low_rank_svd(dt_matrix, singular_count=num_topics)
    term_topic_mat, singular_values, topic_document_mat = u, s, vt

    # Remove singular values below threshold
    min_sigma_value = max(singular_values) * sv_threshold
    singular_values[singular_values < min_sigma_value] = 0

    salience_scores = np.sqrt(np.dot(np.square(s), np.square(vt)))
    top_sentence_indices = (-salience_scores).argsort()[:int(len(sentences) * percentage)]
    top_sentence_indices.sort()

    summary_sentences =('\n'.join(np.array(sentences)[top_sentence_indices]))
    return (summary_sentences)

# Function to perform low-rank SVD
def low_rank_svd(matrix, singular_count=2):
    u, s, vt = svds(matrix, k=singular_count)
    return u, s, vt

# Path to the judgment and summary folders
judgment_folder = '/content/drive/MyDrive/to be delete/judgment'
summary_folder = '/content/drive/MyDrive/to be delete/summary'
excel_file_path = '/content/drive/MyDrive/to be delete/getreadings.xlsx'
df = pd.read_excel(excel_file_path)

# Initialize lists to store LSA evaluation metrics
lsa_rouge1 = []
lsa_rouge2 = []
lsa_rougel = []
lsa_precision = []
lsa_recall = []

# Initialize Rouge object for evaluation
rouge = Rouge()

# Iterate through judgment files and generate LSA summaries
for judgment_file in os.listdir(judgment_folder):
    # Read judgment text from file
    with open(os.path.join(judgment_folder, judgment_file), 'r', encoding='utf-8') as file:
        judgment_text = file.read()

    # Read corresponding reference summary from file
    with open(os.path.join(summary_folder, judgment_file), 'r', encoding='utf-8') as file:
        reference_summary = file.read()

    # Generate LSA summary
    lsa_summary = perform_lsa(nltk.sent_tokenize(judgment_text))

    # Calculate ROUGE scores for LSA summary
    rouge_scores_lsa = rouge.get_scores(lsa_summary, reference_summary)[0]

    # Check if the list has enough elements before accessing specific indices
    if len(sentences) >= 74:  # Check against the problematic index (73)
        # Append metrics to the respective lists
        lsa_rouge1.append(rouge_scores_lsa['rouge-1']['f'])
        lsa_rouge2.append(rouge_scores_lsa['rouge-2']['f'])
        lsa_rougel.append(rouge_scores_lsa['rouge-l']['f'])
        lsa_precision.append(rouge_scores_lsa['rouge-l']['p'])
        lsa_recall.append(rouge_scores_lsa['rouge-l']['r'])
    else:
        print(f"Skipping judgment file: {judgment_file} due to insufficient sentences.")

# Create a DataFrame to store evaluation results
df_lsa = pd.DataFrame({
    '': lsa_rouge1,
    'LSA_ROUGE2': lsa_rouge2,
    'LSA_ROUGEL': lsa_rougel,
    'LSA_Precision': lsa_precision,
    'LSA_Recall': lsa_recall
})

df['BART_ROUGE1'] = rouge1_scores
df['BART_ROUGE2'] = rouge2_scores
df['BART_ROUGEL'] = rougeL_scores
df['BART_Precision'] = precision_scores
df['BART_Recall'] = recall_scores
# Save the LSA evaluation metrics to the Excel file
df_lsa.to_excel(excel_file_path, index=False)

print(f"LSA evaluation metrics added to {excel_file_path}")


(343, 60)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


IndexError: ignored

In [ ]:
lsa_summary

'(j) petitioner: collector land acquisition, anantnag & anr. civil appellate jurisdiction: civil appeal no. aquil shakeel ahmed respondents. (j) bench: thakkar, m.p. general s.k. 1. ordinarily litigant stand benefit lodging appeal late. 460 1987. judgment order dated 14.4. whether apply standard applying "sufficient cause" test litigants regardless personality said context another. appeal preferred state jammu & kashmir arising decision enhancing compensation respect acquisition lands public purpose extent nearly 14 lakhs rupees making upward revision order 800% (from rs. date judgment19/02/1987 bench: thakkar, m.p. order court delivered thakkar, j. condone, condone, question. common knowledge court making justifiably liberal approach matters instituted court. vs. respondent: mst. hence appeal special leave. message appear percolated courts hierarchy. katiji & ors 19 february, 1987 indian kanoon - http://indiankanoon.org/doc/1117226/ 2"any appeal application, application provisions ord

## **LSA Reviewed (This is LSA not Letsum)**

In [ ]:
pip install scikit-learn

In [ ]:
import nltk
import numpy as np
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse.linalg import svds

nltk.download('punkt')
nltk.download('stopwords')

stop_words = nltk.corpus.stopwords.words('english')

def normalize_document(doc):
    doc = re.sub(r'[^a-zA-Z\s]', '', doc, re.I | re.A)
    doc = doc.lower().strip()
    tokens = nltk.word_tokenize(doc)
    filtered_tokens = [token for token in tokens if token not in stop_words]
    doc = ' '.join(filtered_tokens)
    return doc

def low_rank_svd(matrix, singular_count=2):
    u, s, vt = svds(matrix, k=singular_count)
    return u, s, vt

def generate_letsum_summary(text, num_topics=3, percentage_of_sentences=0.5):
    sentences = nltk.sent_tokenize(text)
    norm_sentences = [normalize_document(sentence) for sentence in sentences]

    tv = TfidfVectorizer(min_df=0., max_df=1., use_idf=True)
    dt_matrix = tv.fit_transform(norm_sentences)
    dt_matrix = dt_matrix.toarray()

    vocab = tv.get_feature_names_out()
    td_matrix = dt_matrix.T

    u, s, vt = low_rank_svd(td_matrix, singular_count=num_topics)

    sv_threshold = 0.5
    min_sigma_value = max(s) * sv_threshold
    s[s < min_sigma_value] = 0

    salience_scores = np.sqrt(np.dot(np.square(s), np.square(vt)))

    num_sentences = int(len(sentences) * percentage_of_sentences)
    if len(sentences) < num_sentences:
        num_sentences = len(sentences)

    top_sentence_indices = (-salience_scores).argsort()[:num_sentences]
    top_sentence_indices.sort()

    letsum_summary = ' '.join(np.array(sentences)[top_sentence_indices])
    return letsum_summary


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
import os
import pandas as pd
from rouge import Rouge

# Path to the judgment and summary folders
judgment_folder = '/content/drive/MyDrive/to be delete/judgment'
summary_folder = '/content/drive/MyDrive/to be delete/summary'
excel_file_path = '/content/drive/MyDrive/to be delete/getreadings.xlsx'

# Load existing DataFrame from the Excel file
df = pd.read_excel(excel_file_path)

# Initialize lists to store LetSum evaluation metrics
letsum_rouge1 = []
letsum_rouge2 = []
letsum_rougel = []
letsum_precision = []
letsum_recall = []
letsum_f1_scores = []

# Initialize Rouge object for evaluation
rouge = Rouge()

# Iterate through judgment files and generate LetSum summaries
for judgment_file in os.listdir(judgment_folder):
    # Read judgment text from file
    with open(os.path.join(judgment_folder, judgment_file), 'r', encoding='utf-8') as file:
        judgment_text = file.read()

    # Read corresponding summary text from file
    with open(os.path.join(summary_folder, judgment_file), 'r', encoding='utf-8') as file:
        reference_summary = file.read()

    # Generate LetSum summary
    letsum_summary = generate_letsum_summary(judgment_text)
    letsum_summary= letsum_summary.replace(u'\xa0',' ')
    letsum_summary= letsum_summary.replace(u'\xad',' ')
    letsum_summary= letsum_summary.replace(u'\n',' ')


    # Calculate ROUGE scores for LetSum summary
    rouge_scores_letsum = rouge.get_scores(letsum_summary, reference_summary)[0]

    # Append metrics to the respective lists
    letsum_rouge1.append(rouge_scores_letsum['rouge-1']['f'])
    letsum_rouge2.append(rouge_scores_letsum['rouge-2']['f'])
    letsum_rougel.append(rouge_scores_letsum['rouge-l']['f'])
    letsum_precision.append(rouge_scores_letsum['rouge-l']['p'])
    letsum_recall.append(rouge_scores_letsum['rouge-l']['r'])
    letsum_f1_scores.append(rouge_scores_letsum['rouge-l']['f'])

# Add LetSum evaluation metrics to the DataFrame
df['LetSum_ROUGE1'] = letsum_rouge1
df['LetSum_ROUGE2'] = letsum_rouge2
df['LetSum_ROUGEL'] = letsum_rougel
df['LetSum_Precision'] = letsum_precision
df['LetSum_Recall'] = letsum_recall
df['LetSum_F1'] = letsum_f1_scores

# Save the updated DataFrame to the Excel file
df.to_excel(excel_file_path, index=False)

print(f"LetSum evaluation metrics added to {excel_file_path}")


In [ ]:
letsum_summary

'supreme court of india collector land acquisition,   vs mst. katiji & ors on 19 february, 1987 equivalent citations: 1987 air 1353, 1987 scr (2) 387 author: m thakkar bench: thakkar, m.p. (j)            petitioner: collector land acquisition, anantnag & anr. vs. respondent: mst. katiji & ors. date of judgment19/02/1987 bench: thakkar, m.p. (j) bench: thakkar, m.p. civil appellate jurisdiction: civil appeal no. 1986 of the jammu & and kashmir high court in civil 1st appeal no. the order of the court was delivered by thakkar, j. to condone, or not to condone, is not the only question. an appeal preferred by the state of jammu & kashmir arising out of a decision enhancing compensation in respect of acquisition of lands for a public purpose to the extent of nearly 14 lakhs rupees by making an upward revision of the order of 800% (from rs. the legislature has conferred the power to condone delay by enacting section 51 of the indian limitation act of 1963 in order to enable the courts to do

## **BERT**

In [ ]:
pip install bert-extractive-summarizer

In [ ]:
import os
import pandas as pd
from summarizer import Summarizer
from rouge import Rouge

# Initialize BERT-based extractive summarizer
model = Summarizer()

# Path to the judgment and summary folders
judgment_folder = '/content/drive/MyDrive/Headnote Judgment/Judgment'
summary_folder = '/content/drive/MyDrive/Headnote Judgment/Summary'
excel_file_path = '/content/drive/MyDrive/to be delete/getreadings.xlsx'

# Initialize lists to store BERT evaluation metrics
bert_rouge1 = []
bert_rouge2 = []
bert_rougel = []
bert_precision = []
bert_recall = []

# Initialize Rouge object for evaluation
rouge = Rouge()

# Iterate through judgment files and generate BERT summaries
for judgment_file in os.listdir(judgment_folder):
    # Read judgment text from file
    with open(os.path.join(judgment_folder, judgment_file), 'r', encoding='utf-8') as file:
        judgment_text = file.read()

    # Generate BERT summary
    bert_summary = model(judgment_text)

    # Read corresponding reference summary from file
    with open(os.path.join(summary_folder, judgment_file), 'r', encoding='utf-8') as file:
        reference_summary = file.read()

    # Calculate ROUGE scores for BERT summary
    rouge_scores_bert = rouge.get_scores(bert_summary, reference_summary)[0]

    # Append metrics to the respective lists
    bert_rouge1.append(rouge_scores_bert['rouge-1']['f'])
    bert_rouge2.append(rouge_scores_bert['rouge-2']['f'])
    bert_rougel.append(rouge_scores_bert['rouge-l']['f'])
    bert_precision.append(rouge_scores_bert['rouge-l']['p'])
    bert_recall.append(rouge_scores_bert['rouge-l']['r'])

# Create a DataFrame to store evaluation results
df_bert = pd.DataFrame({
    'BERT_ROUGE1': bert_rouge1,
    'BERT_ROUGE2': bert_rouge2,
    'BERT_ROUGEL': bert_rougel,
    'BERT_Precision': bert_precision,
    'BERT_Recall': bert_recall
})

# Save the BERT evaluation metrics to the Excel file
df_bert.to_excel(excel_file_path, index=False)

print(f"BERT evaluation metrics added to {excel_file_path}")


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:8

BERT evaluation metrics added to /content/drive/MyDrive/to be delete/getreadings.xlsx


In [ ]:
bert_summary

NameError: ignored

In [ ]:
from sklearn.metrics import cohen_kappa_score
kappa_score = cohen_kappa_score(true_labels, predicted_labels)